In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import logging

from conda_forge_tick.utils import setup_logging

setup_logging("DEBUG")

## Run A Migration

In [3]:
from conda_forge_tick.all_feedstocks import get_all_feedstocks
from conda_forge_tick.utils import load_graph
from conda_forge_tick.feedstock_parser import load_feedstock

gx = load_graph()

In [4]:
from conda_forge_tick.make_migrators import load_migrators

mgs = load_migrators()

loading migrators: 100%|████████████████████████| 78/78 [00:14<00:00,  5.50it/s]


In [5]:
for mg in mgs:
    if "boost" in mg.name:
        print(mg.name)
        break

libboost186


In [9]:
from conda_forge_tick.migration_runner import run_migration_local

In [11]:
with gx.nodes["basictools"]["payload"] as attrs:

    data = run_migration_local(
        migrator=mg,
        feedstock_dir="../basictools-feedstock",
        feedstock_name="basictools",
        node_attrs=attrs,
        default_branch="main",
    )

In [ ]:
from conda_forge_feedstock_check_solvable import is_recipe_solvable

In [ ]:
res = is_recipe_solvable(
    "../hpp-fcl-feedstock",
    build_platform=dict(
      linux_aarch64="linux_64",
      linux_ppc64le="linux_64",
      osx_arm64="osx_64",
    ),
    verbosity=3,
    solver="mamba",
)
        
        

In [ ]:
import json

with open("/Users/beckermr/Downloads/repodata.json") as fp:
    rd = json.load(fp)

In [ ]:
for key in rd["packages.conda"]:
    if key.startswith("qhull-static") or key.startswith("libboost-python-devel"):
        print(key)

## Build a Node

In [5]:
from conda_forge_tick.all_feedstocks import get_all_feedstocks
from conda_forge_tick.utils import load_graph
from conda_forge_tick.feedstock_parser import load_feedstock

gx = load_graph()


In [6]:
from conda_forge_tick.make_migrators import load_migrators

mgs = load_migrators(skip_paused=False)

loading migrators: 100%|████████████████████████| 66/66 [00:11<00:00,  5.68it/s]


In [7]:
dmgs = {mg.name: mg for mg in mgs}
assert len(dmgs) == len(mgs)

mg = dmgs["python313"]

In [9]:
import networkx as nx

gg = mg.graph

for cyc in nx.chordless_cycles(gg):
    print(cyc)

gg.remove_edges_from(nx.selfloop_edges(gg))

for cyc in nx.chordless_cycles(gg):
    print(cyc)

['gnuradio', 'gnuradio-soapy']
['heavydb-ext', 'omniscidb']
['fenics-basix', 'fenics-basix-meta']
['intel-compiler-repack', 'numpy', 'pyopencl']
['gnuradio', 'gnuradio-soapy']
['heavydb-ext', 'omniscidb']
['fenics-basix', 'fenics-basix-meta']
['intel-compiler-repack', 'numpy', 'pyopencl']


In [10]:
# gx.remove_edges_from(nx.selfloop_edges(gx))

cyc = ['intel-compiler-repack', 'numpy', 'pyopencl']
for node in cyc:
    print(
        node, "\n", 
        "pred:", [cy for cy in cyc if cy in list(gg.predecessors(node))], "\n", 
        "succ:", [cy for cy in cyc if cy in list(gg.successors(node))]
    )

intel-compiler-repack 
 pred: ['pyopencl'] 
 succ: ['numpy']
numpy 
 pred: ['intel-compiler-repack'] 
 succ: ['pyopencl']
pyopencl 
 pred: ['numpy'] 
 succ: ['intel-compiler-repack']


In [11]:
gx.nodes['intel-compiler-repack']["payload"]["requirements"]["host"]

{'_openmp_mutex',
 'c_compiler_stub',
 'c_stdlib_stub',
 'ctng-compiler-activation',
 'ctng-compilers',
 'cxx_compiler_stub',
 'dpcpp-cpp-rt',
 'intel-cmplr-lib-rt',
 'intel-cmplr-lib-ur',
 'intel-cmplr-lic-rt',
 'intel-compiler-repack',
 'intel-opencl-rt',
 'intel-sycl-rt',
 'khronos-opencl-icd-loader',
 'level-zero-devel',
 'libxml2',
 'linux-sysroot',
 'llvm-openmp',
 'mpich',
 'ocl-icd',
 'openmp',
 'python',
 'tbb-devel',
 'vc',
 'zlib'}

In [ ]:
with gx.nodes["gdal"]["payload"] as attrs:
    attrs = load_feedstock("gdal", attrs)

In [ ]:
list(attrs.keys()), attrs["platforms"]

## Debug a Version Migration

In [ ]:
from conda_forge_tick.make_migrators import load_migrators

In [ ]:
mgs = load_migrators()

In [ ]:
from conda_forge_tick.all_feedstocks import get_all_feedstocks
from conda_forge_tick.utils import load_graph

gx = load_graph()

In [ ]:
from conda_forge_tick.utils import change_log_level

with change_log_level("conda_forge_tick", "DEBUG"):
    print(mgs[0].filter(gx.nodes["hstream"]["payload"].data))

## Find Some Files

In [ ]:
from conda_forge_tick.lazy_json_backends import get_sharded_path

In [ ]:
get_sharded_path("node_attrs/ucx-split.json")

In [ ]:
from conda_forge_tick.lazy_json_backends import LazyJson

with LazyJson("node_attrs/ucx-split.json") as attrs:
    print(attrs.data.get("version", None))

In [ ]:
from conda_forge_tick.feedstock_parser import load_feedstock_local

attrs = {}
load_feedstock_local("ucx-split", attrs)

In [ ]:
attrs["version"]

## Load the graph

In [ ]:
from conda_forge_tick.all_feedstocks import get_all_feedstocks
from conda_forge_tick.utils import load_graph

gx = load_graph()

In [ ]:
import pprint
import yaml

node = "setuptools_scm"

print(yaml.dump(gx.nodes[node]["payload"].data["requirements"]))

## Check Some Version Stuff

In [ ]:
from conda_forge_tick.update_upstream_versions import update_upstream_versions

In [ ]:
import copy

gxc = copy.deepcopy(gx)

for node in list(gxc.nodes):
    if node not in ["tzdata", "openssl", "jpeg", "cddlib"]:
        gxc.remove_node(node)

In [ ]:
update_upstream_versions(gxc, debug=True)

## Look at a Migration

In [ ]:
from conda_forge_tick.auto_tick import migration_factory, add_arch_migrate

In [ ]:
mgs = []
add_arch_migrate(mgs, gx)

In [ ]:
mg = mgs[0]

In [ ]:
node_to_check = "maturin"

In [ ]:
from conda_forge_tick.make_graph import get_deps_from_outputs_lut
import networkx as nx

arches = {
    "linux_aarch64": "default",
    "linux_ppc64le": "default",
}


graph2 = nx.create_empty_copy(gx)
for node, attrs in gx.nodes(data="payload"):
    for plat_arch in arches:
        deps = set().union(
            *attrs.get(
                f"{plat_arch}_requirements",
                attrs.get("requirements", {}),
            ).values()
        )
        for dep in get_deps_from_outputs_lut(deps, gx.graph["outputs_lut"]):
            graph2.add_edge(dep, node)

In [ ]:
print(sorted(graph2.predecessors(node_to_check)))

In [ ]:
from conda_forge_tick.utils import pluck

graph = graph2
packages = mg.target_packages.copy()
for target in mg.target_packages:
    if target in graph.nodes:
        packages.update(nx.ancestors(graph, target))
for node in list(graph.nodes.keys()):
    if node not in packages:
        pluck(graph, node)
# post-plucking cleanup
graph.remove_edges_from(nx.selfloop_edges(graph))

In [ ]:
print(sorted(graph2.predecessors(node_to_check)))

In [ ]:
from conda_forge_tick.migrators.migration_yaml import all_noarch

ignored_packages = {
    "make",
    "perl",
    "toolchain",
    "posix",
    "patchelf",  # weird issue
}

latest_preds = sorted(graph.predecessors(node_to_check))

for node, attrs in list(graph.nodes("payload")):
    if (
        (not attrs)
        or node.endswith("_stub")
        or node.startswith("m2-")
        or node.startswith("m2w64-")
        or node.startswith("__")
        or (node in ignored_packages)
        or all_noarch(attrs)
    ):
        pluck(graph, node)
        new_latest_preds = sorted(graph.predecessors(node_to_check))
        if new_latest_preds != latest_preds:
            print(node)
            print("    new preds:", new_latest_preds)
            print("    old preds:", latest_preds)
            print(" ")
            latest_preds = new_latest_preds

# post-plucking cleanup
graph.remove_edges_from(nx.selfloop_edges(graph))

In [ ]:
print(sorted(graph2.predecessors(node_to_check)))

In [ ]:
import os
from conda_forge_tick.contexts import MigratorContext, MigratorSessionContext


mctx = MigratorSessionContext(
    circle_build_url=os.getenv("CIRCLE_BUILD_URL", ""),
    graph=gx,
    smithy_version="",
    pinning_version="",
    github_username="",
    github_password="",
    github_token="",
    dry_run=False,
)

mmctx = MigratorContext(session=mctx, migrator=mg)
mg.bind_to_ctx(mmctx)

In [ ]:
mg.graph["outputs_lut"]

In [ ]:
import networkx as nx

In [ ]:
print(sorted(mg.graph.predecessors("bcrypt")))

In [ ]:
mgs = []
add_rebuild_broken_migrator(mgs, gx)

In [ ]:
mg = mgs[0]

In [ ]:
import os
from conda_forge_tick.contexts import MigratorContext, MigratorSessionContext


mctx = MigratorSessionContext(
    circle_build_url=os.getenv("CIRCLE_BUILD_URL", ""),
    graph=gx,
    smithy_version="",
    pinning_version="",
    github_username="",
    github_password="",
    github_token="",
    dry_run=False,
)

mmctx = MigratorContext(session=mctx, migrator=mg)
mg.bind_to_ctx(mmctx)

In [ ]:
mmctx.effective_graph.nodes

In [ ]:
mg_name = "libffi33"

mgs = []
migration_factory(mgs, gx, only_keep=[mg_name])

for i in range(len(mgs)):
    if mgs[i].name == mg_name:
        break
mg = mgs[i]

In [ ]:
import copy

attrs = copy.deepcopy(mg.graph.nodes["python"]["payload"].data)

attrs["branch"] = "3.6"
mg.filter(attrs)

In [ ]:
import os
from conda_forge_tick.contexts import MigratorContext, MigratorSessionContext


mctx = MigratorSessionContext(
    circle_build_url=os.getenv("CIRCLE_BUILD_URL", ""),
    graph=gx,
    smithy_version="",
    pinning_version="",
    github_username="",
    github_password="",
    github_token="",
)

mmctx = MigratorContext(session=mctx, migrator=mg)
mg.bind_to_ctx(mmctx)


In [ ]:
mmctx.effective_graph.nodes

## Check the Status Report

In [ ]:
from conda_forge_tick.status_report import  graph_migrator_status

In [ ]:
out2, build_sequence, gv = graph_migrator_status(mg, mg.graph)

In [ ]:
gv.view()

In [ ]:
build_sequence

In [ ]:
len(build_sequence)

In [ ]:
gx.nodes["proj"]["payload"].data

## Debug Pinning Migrations

In [ ]:
from conda_forge_tick.auto_tick import create_migration_yaml_creator

In [ ]:
mgs = []
create_migration_yaml_creator(mgs, gx, pin_to_debug="ucx")

In [ ]:
gx.nodes["ucx-split"]["payload"].get("version")